In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
from statsmodels.stats.outliers_influence import variance_inflation_factor # VIF
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
df = pd.read_csv('https://raw.githubusercontent.com/ryupepa/Econometrica/master/python-notebooks/data-csv/sleep75.csv')
df['age_2']=df['age']*df['age']
df

,age,black,case,clerical,construc,educ,earns74,gdhlth,inlf,leis1,...,totwrk,union,worknrm,workscnd,exper,yngkid,yrsmarr,hrwage,agesq,age_2
0,32,0,1,0.000000,0.000000,12,0,0,1,3529,...,3438,0,3438,0,14,0,13,7.070004,1024,1024
1,31,0,2,0.000000,0.000000,14,9500,1,1,2140,...,5020,0,5020,0,11,0,0,1.429999,961,961
2,44,0,3,0.000000,0.000000,17,42500,1,1,4595,...,2815,0,2815,0,21,0,0,20.529997,1936,1936
3,30,0,4,0.000000,0.000000,12,42500,1,1,3211,...,3786,0,3786,0,12,0,12,9.619998,900,900
4,64,0,5,0.000000,0.000000,14,2500,1,1,4052,...,2580,0,2580,0,44,0,33,2.750000,4096,4096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,45,0,702,0.182331,0.030075,12,5500,1,0,5069,...,2026,0,2026,0,27,0,18,NaN,2025,2025
702,34,0,703,0.182331,0.030075,10,2500,0,0,5885,...,675,1,465,210,18,0,4,NaN,1156,1156
703,37,0,704,0.182331,0.030075,12,3500,1,0,4719,...,1851,0,1851,0,19,0,17,NaN,1369,1369
704,54,0,705,0.182331,0.030075,17,32500,1,0,5149,...,1961,1,1481,480,31,0,22,NaN,2916,2916


In [10]:
# Модель sleep на totwrk/100, totwrk^2/10000, age, smsa, male
Sleep_eq12 = smf.ols(formula = 'sleep~totwrk +  age + +age_2+ smsa + male + south', data = df).fit()


In [11]:
info_dict = {'No. Observations': lambda x: '{0:d}'.format(int(x.nobs)), 
             'Residual Std. Error': lambda x: "%#8.3f" % x.mse_resid**0.5,
             'F-statistic': lambda x: '{:.3f}'.format(x.fvalue), 
             'Prob (F-statistic)': lambda x: '{:.3f}'.format(x.f_pvalue)
             }
print(summary_col(Sleep_eq12, float_format='%.3f', stars=True, info_dict = info_dict))


                       sleep   
-------------------------------
Intercept           3638.626***
                    (218.595)  
totwrk              -0.168***  
                    (0.018)    
age                 -6.165     
                    (11.176)   
age_2               0.108      
                    (0.134)    
smsa                -56.699*   
                    (32.931)   
male                90.972***  
                    (34.210)   
south               97.640**   
                    (41.752)   
R-squared           0.131      
R-squared Adj.      0.124      
No. Observations    706        
Residual Std. Error 415.975    
F-statistic         17.615     
Prob (F-statistic)  0.000      
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [12]:
df = summary_params(Sleep_eq12, alpha=0.1)
df['significance'] = df.apply(lambda x: 'Значим' if x['P>|t|']<0.1 else 'Незначим', axis=1)
df

,Coef.,Std.Err.,t,P>|t|,[0.05,0.95],significance
Intercept,3638.626269,218.595464,16.645479,1.181161e-52,3278.591570,3998.660968,Значим
totwrk,-0.168165,0.018113,-9.283974,2.018779e-19,-0.197998,-0.138331,Значим
age,-6.165070,11.175631,-0.551653,5.813624e-01,-24.571741,12.241601,Незначим
age_2,0.108344,0.133550,0.811266,4.174889e-01,-0.111617,0.328305,Незначим
smsa,-56.698990,32.931132,-1.721744,8.555826e-02,-110.937765,-2.460215,Значим
male,90.971527,34.210484,2.659171,8.012687e-03,34.625612,147.317441,Значим
south,97.639593,41.751613,2.338583,1.963799e-02,28.873165,166.406022,Значим
